#   Torch IO library for 3-D images

##   TorchIO Image class

> The Image class, representing one medical image, stores a 4D tensor, whose voxels encode, e.g., signal intensity or segmentation labels, and the corresponding affine transform, typically a rigid (Euclidean) transform, to convert voxel indices to world coordinates in mm. Arbitrary fields such as acquisition parameters may also be stored.

> Subclasses are used to indicate specific types of images, such as ScalarImage and LabelMap, which are used to store, e.g., CT scans and segmentations, respectively.

> An instance of Image can be created using a filepath, a PyTorch tensor, or a NumPy array. This class uses lazy loading, i.e., the data is not loaded from disk at instantiation time. Instead, the data is only loaded when needed for an operation (e.g., if a transform is applied to the image).

> The figure below shows two instances of Image. The instance of ScalarImage contains a 4D tensor representing a diffusion MRI, which contains four 3D volumes (one per gradient direction), and the associated affine matrix. Additionally, it stores the strength and direction for each of the four gradients. The instance of LabelMap contains a brain parcellation of the same subject, the associated affine matrix, and the name and color of each brain structure.

<img src='https://torchio.readthedocs.io/_images/data_structures.png'>

In [ ]:
import torch
import torchio as tio

# Loading from a file
t1_image = tio.ScalarImage('t1.nii.gz')
dmri = tio.ScalarImage(tensor=torch.rand(32, 128, 128, 88))
image = tio.ScalarImage('safe_image.nrrd', check_nans=False)
data, affine = image.data, image.affine

print (affine.shape)
print (image.data is image[tio.DATA])
print (image.data is image.tensor )
print (type(image.data) )

> Intensity transforms are not applied to these images.

> Nearest neighbor interpolation is always used to resample label maps, independently of the specified interpolation type in the transform instantiation.

In [ ]:
import torch
import torchio as tio

labels = tio.LabelMap(tensor=torch.rand(1, 128, 128, 68) > 0.5)
labels = tio.LabelMap('t1_seg.nii.gz')  
tpm = tio.LabelMap('gray_matter.nii.gz', 'white_matter.nii.gz', 'csf.nii.gz')

<img src='images/image_class.png'>

<img src='images/parameters.png'>

In [ ]:
import torchio as tio
import numpy as np

image = tio.ScalarImage('t1.nii.gz')  # subclass of Image
image  # not loaded yet

In [ ]:
times_two = 2 * image.data  # data is loaded and cached here
print (image)

image.save('doubled_image.nii.gz')

In [ ]:
# Defining a function
def numpy_reader(path):
    data = np.load(path).as_type(np.float32)
    affine = np.eye(4)
    return data, affine

image = tio.ScalarImage('t1.npy', reader=numpy_reader)

In [ ]:
##  properties

In [ ]:
 => affine: numpy.ndarray => Affine matrix to transform voxel indices into world coordinates.
        
 => as_pil(transpose=True) => Get the image as an instance of PIL.Image.

     ** Note => Values will be clamped to 0-255 and cast to uint8 and 
                    To use this method, Pillow needs to be installed.

In [ ]:
 => as_sitk(**kwargs) → SimpleITK.SimpleITK.Image  => Get the image as an instance of sitk.Image.

In [ ]:
 => axis_name_to_index(axis: str) => int  =>  Convert an axis name to an axis index.
    
   ** axis – Possible inputs are 'Left', 'Right', 'Anterior', 'Posterior', 'Inferior', 'Superior'. 
      
   ** Lower-case versions and first letters are also valid, as only the first letter will be used.

 => If you are working with animals, you should probably use 'Superior', 'Inferior', 'Anterior' and 'Posterior'
     for 'Dorsal', 'Ventral', 'Rostral' and 'Caudal', respectively.
    
 => If your images are 2D, you can use 'Top', 'Bottom', 'Left' and 'Right'.

In [ ]:
 => bounds: numpy.ndarray  =>  Position of centers of voxels in smallest and largest indices.

 => data: torch.Tensor  =>  Tensor data (Same as Image.tensor)
    
 => static flip_axis(axis: str) → str[source]  => Return the opposite axis label. For example, 'L' -> 'R'.

  ** axis – Axis label, such as 'L' or 'left'.

In [ ]:
 =>  classmethod from_sitk(sitk_image)  =>  Instantiate a new TorchIO image from a sitk.Image.

In [ ]:
import torchio as tio
import SimpleITK as sitk

sitk_image = sitk.Image(20, 30, 40, sitk.sitkUInt16)
tio.LabelMap.from_sitk(sitk_image)
sitk_image = sitk.Image((224, 224), sitk.sitkVectorFloat32, 3)
tio.ScalarImage.from_sitk(sitk_image)

In [ ]:
 =>  get_bounds() => Tuple[Tuple[float, float], Tuple[float, float], Tuple[float, float]]
                          Get minimum and maximum world coordinates occupied by the image.
        
 =>  get_center(lps: bool = False) => Tuple[float, float, float] =>  Get image center in RAS+ or LPS+ coordinates.

    ** lps – If True, the coordinates will be in LPS+ orientation, i.e. the first dimension grows towards
               the left etc. Otherwise, the coordinates will be in RAS+ orientation.

In [ ]:
 =>  height: int  =>  Image height, if 2D.
        
 =>  itemsize =>  Element size of the data type.

 =>  load() => None => Load the image from disk.
    
     Returns => Tuple containing a 4D tensor of size (C,W,H,D) and a 2D(4,4)  affine matrix to convert 
                 voxel indices to world coordinates.

In [ ]:
 =>  memory: float  =>  Number of Bytes that the tensor takes in the RAM.
        
 =>  num_channels: int  => Get the number of channels in the associated 4D tensor.

 =>  numpy() → numpy.ndarray  =>  Get a NumPy array containing the image data.


In [ ]:
 => orientation: Tuple[str, str, str]  =>  Orientation codes.
        
 => origin: Tuple[float, float, float]  => Center of first voxel in array, in mm.

 => plot(**kwargs) → None =>  Plot image.

 => save(path: Union[str, os.PathLike], squeeze: Optional[bool] = None) => None  =>  Save image to disk.

      ** path – String or instance of pathlib.Path.

      ** squeeze – Whether to remove singleton dimensions before saving. 
                     If None, the array will be squeezed if the output format is JP(E)G, PNG, BMP or TIF(F).


 => set_data(tensor: Union[torch.Tensor, numpy.ndarray]) =>  Store a 4D tensor in the data key and attribute.

     ** tensor – 4D tensor (C,W,H,D) with dimensions .

 => shape: Tuple[int, int, int, int] => Tensor shape as (C,W,H,D).

In [ ]:
 => show(viewer_path: Optional[Union[str, os.PathLike]] = None) => None => Open the image using external software.
    
    **  viewer_path – Path to the application used to view the image. 
                      If None, the value of the environment variable SITK_SHOW_COMMAND will be used. 
                      If this variable is also not set, TorchIO will try to guess the location of ITK-SNAP 
                         and 3D Slicer.  RAISES RuntimeError – If the viewer is not found.
 => spacing: Tuple[float, float, float] => Voxel spacing in mm.
    
 => spatial_shape: Tuple[int, int, int]  =>  Tensor spatial shape as (C,W,H,D) .
    
 => tensor: torch.Tensor =>  Tensor data (Same as Image.data)

In [ ]:
 => to_gif(axis: int, duration: float, output_path: Union[str, os.PathLike], loop: int = 0,
            rescale: bool = True, optimize: bool = True, reverse: bool = False) => None
    
 => to_gif() saves an animated GIF of the image.

<img src="images/to_gif.png">

 ##  Subject Class

<img src='images/subject_class.png'>

In [ ]:
import torchio as tio

# One way:
subject = tio.Subject(one_image=tio.ScalarImage('path_to_image.nii.gz'), 
                      a_segmentation=tio.LabelMap('path_to_seg.nii.gz'),
                      age=45, name='John Doe', hospital='Hospital Juan Negrín')

# If you want to create the mapping before, or have spaces in the keys -
subject_dict = {'one image': tio.ScalarImage('path_to_image.nii.gz'),
                'a segmentation': tio.LabelMap('path_to_seg.nii.gz'),
                'age': 45, 'name': 'John Doe', 'hospital': 'Hospital Juan Negrín'}

subject = tio.Subject(subject_dict)

In [ ]:
 => add_image(image: torchio.data.image.Image, image_name: str) => None  =>  Add an image.

 => apply_inverse_transform(**kwargs) → torchio.data.subject.Subject => Try to apply the inverse of all 
                                                                          applied transforms, in reverse order.

       **kwargs – Keyword arguments passed on to get_inverse_transform().

In [ ]:
##   Check for consistency of an attribute across all images.

=> check_consistent_attribute(attribute: str, relative_tolerance: float = 1e-06, 
                               absolute_tolerance: float = 1e-06, message: Optional[str] = None) → None
    
  ## Parameters  -
   
   * attribute – Name of the image attribute to check

   * relative_tolerance – Relative tolerance for numpy.allclose()

   * absolute_tolerance – Absolute tolerance for numpy.allclose()

In [ ]:
import numpy as np
import torch
import torchio as tio

scalars = torch.randn(1, 512, 512, 100)
mask = torch.tensor(scalars > 0).type(torch.int16)

af1 = np.eye([0.8, 0.8, 2.50000000000001, 1])
af2 = np.eye([0.8, 0.8, 2.49999999999999, 1])  # small difference here (e.g. due to different reader)

subject = tio.Subject(image = tio.ScalarImage(tensor=scalars, affine=af1),
                      mask = tio.LabelMap(tensor=mask, affine=af2))

subject.check_consistent_attribute('spacing')  # no error as tolerances are > 0

<img src='images/relative_tolerance.png'>

In [ ]:
 ## Get a reversed list of the inverses of the applied transforms.
    
=> get_inverse_transform(warn: bool = True, ignore_intensity: bool = True, image_interpolation: 
                           Optional[str] = None) → Compose 
    
 
          Parameters 
    
    * warn – Issue a warning if some transforms are not invertible.

    * ignore_intensity – If True, all instances of IntensityTransform will be ignored.

    * image_interpolation – Modify interpolation for scalar images inside transforms that perform resampling.

In [ ]:
 => load() → None  =>  Load images in subject on RAM.
    
 => plot(**kwargs) → None  =>  Plot images using matplotlib.

        **kwargs – Keyword arguments that will be passed on to plot().

In [ ]:
 => remove_image(image_name: str) => None  =>  Remove an image.
    
 => shape => Return shape of first image in subject.

 ** Note => Consistency of shapes across images in the subject is checked first.

In [ ]:
import torchio as tio

colin = tio.datasets.Colin27()
colin.shape

In [ ]:
 => spacing => Return spacing of first image in subject.

  ** Note =>  Consistency of spacings across images in the subject is checked first.

In [ ]:
import torchio as tio

colin = tio.datasets.Slicer()
colin.shape

In [ ]:
 => spatial_shape  =>  Return spatial shape of first image in subject.

  ** Note =>  Consistency of spatial shapes across images in the subject is checked first.

In [ ]:
import torchio as tio

colin = tio.datasets.Colin27()
colin.shape

##  DATASET Class

<img src='https://torchio.readthedocs.io/_images/diagram_volumes.svg' />
</br>
<img src='images/dataset.png' />

In [ ]:
import torchio as tio
subject_a = tio.Subject(
    t1=tio.ScalarImage('t1.nrrd',),
    t2=tio.ScalarImage('t2.mha',),
    label=tio.LabelMap('t1_seg.nii.gz'),
    age=31,
    name='Fernando Perez',
)
subject_b = tio.Subject(
    t1=tio.ScalarImage('colin27_t1_tal_lin.minc',),
    t2=tio.ScalarImage('colin27_t2_tal_lin_dicom',),
    label=tio.LabelMap('colin27_seg1.nii.gz'),
    age=56,
    name='Colin Holmes',
)
subjects_list = [subject_a, subject_b]
transforms = [
    tio.RescaleIntensity(out_min_max=(0, 1)),
    tio.RandomAffine(),
]
transform = tio.Compose(transforms)
subjects_dataset = tio.SubjectsDataset(subjects_list, transform=transform)
subject = subjects_dataset[0]



In [ ]:
 => To quickly iterate over the subjects without loading the images, use dry_iter().

 =>  dry_iter()  =>  Return the internal list of subjects.

 => This can be used to iterate over the subjects without loading the data and applying any transforms

In [ ]:
names = [subject.name for subject in dataset.dry_iter()]

In [ ]:
 => classmethod from_batch(batch: Dict) → torchio.data.dataset.SubjectsDataset
    
 => Instantiates a dataset from a batch generated by a data loader.

    ** batch – Dictionary generated by a data loader, containing data that can be converted to 
                  instances of Subject.

In [ ]:
 =>  set_transform(transform: Optional[Callable]) =>  None  =>  Set the transform attribute
    
    ** transform – Callable object, typically an subclass of torchio.transforms.Transform.